In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import random as rnd
import math
from pytrends.request import TrendReq
from seaborn import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, classification_report

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

kw_list = ["Tesla"] 

pytrends.build_payload(kw_list, timeframe='2021-03-31 2021-07-22', geo='US')

Trendsdf = pytrends.interest_over_time()
Trendsdf = Trendsdf.rename(columns={'Tesla': 'Previous_Search_Interest'})
#Trendsdf.reset_index(inplace=True, drop=True)
Trendsdf.shape
Trendsdf

,Previous_Search_Interest,isPartial
date,,
2021-03-31,71,False
2021-04-01,76,False
2021-04-02,71,False
2021-04-03,61,False
2021-04-04,64,False
...,...,...
2021-07-17,59,False
2021-07-18,58,False
2021-07-19,65,False


In [3]:
stockPricePrevious = yf.download("TSLA", start="2021-03-31", end="2021-07-22")
stockPricePrevious = stockPricePrevious.rename(columns={'Open': 'Previous_Open', 'Close': 'Previous_Close','High': 'Previous_High','Volume': 'Previous_Volume','Low': 'Previous_Low', 'Adj Close':'Previous_Adj_Close'})
stockPricePrevious

mergedPriceTrends = pd.concat([stockPricePrevious, Trendsdf], axis=1)
mergedPriceTrends.reset_index(inplace=False, drop=True)
mergedPriceTrends.dropna(inplace=True)
#mergedPriceTrends.reset_index()


#print(mergedPriceTrends)
stockPrice = yf.download("TSLA", start="2021-04-01", end="2021-07-23")
#stockPrice.reset_index(inplace=True, drop=True)
#print(stockPrice)
mergedPrice = pd.concat([mergedPriceTrends, stockPrice], axis=1)
#mergedPrice['Previous_Open', 'Previous_High','Previous_Low','Previous_Close', 'Previous_Adj_Close', 'Previous_Volume'] = mergedPrice['Previous_Open', 'Previous_High','Previous_Low','Previous_Close', 'Previous_Adj_Close', 'Previous_Volume'].shift(1)
mergedPrice['Previous_Open'] = mergedPrice['Previous_Open'].shift(1)
mergedPrice['Previous_High'] = mergedPrice['Previous_High'].shift(1)
mergedPrice['Previous_Low'] = mergedPrice['Previous_Low'].shift(1)
mergedPrice['Previous_Close'] = mergedPrice['Previous_Close'].shift(1)
mergedPrice['Previous_Adj_Close'] = mergedPrice['Previous_Adj_Close'].shift(1)
mergedPrice['Previous_Volume'] = mergedPrice['Previous_Volume'].shift(1)
mergedPrice['Previous_Search_Interest'] = mergedPrice['Previous_Search_Interest'].shift(1)
mergedPrice.dropna(inplace=True)
#mergedPrice.set_index('isPartial', drop=False)
mergedPrice
#mergedPriceTrends

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Adj_Close,Previous_Volume,Previous_Search_Interest,isPartial,Open,High,Low,Close,Adj Close,Volume
2021-04-01,646.619995,672.000000,641.109985,667.929993,667.929993,33337300.0,71.0,False,688.369995,692.419983,659.419983,661.750000,661.750000,35298400.0
2021-04-05,688.369995,692.419983,659.419983,661.750000,661.750000,35298400.0,76.0,False,707.710022,708.159973,684.700012,691.049988,691.049988,41842800.0
2021-04-06,707.710022,708.159973,684.700012,691.049988,691.049988,41842800.0,82.0,False,690.299988,696.549988,681.369995,691.619995,691.619995,28271800.0
2021-04-07,690.299988,696.549988,681.369995,691.619995,691.619995,28271800.0,69.0,False,687.000000,691.380005,667.840027,670.969971,670.969971,26309400.0
2021-04-08,687.000000,691.380005,667.840027,670.969971,670.969971,26309400.0,64.0,False,677.380005,689.549988,671.650024,683.799988,683.799988,23924300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-15,670.750000,678.609985,652.840027,653.380005,653.380005,21641200.0,57.0,False,658.390015,666.140015,637.880005,650.599976,650.599976,20209600.0
2021-07-16,658.390015,666.140015,637.880005,650.599976,650.599976,20209600.0,59.0,False,654.679993,656.700012,642.200012,644.219971,644.219971,16339800.0
2021-07-19,654.679993,656.700012,642.200012,644.219971,644.219971,16339800.0,62.0,False,629.890015,647.200012,621.289978,646.219971,646.219971,21297100.0
2021-07-20,629.890015,647.200012,621.289978,646.219971,646.219971,21297100.0,65.0,False,651.989990,662.390015,640.500000,660.500000,660.500000,15442700.0


In [4]:
mergedPrice.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 77 entries, 2021-04-01 to 2021-07-21
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Previous_Open             77 non-null     float64
 1   Previous_High             77 non-null     float64
 2   Previous_Low              77 non-null     float64
 3   Previous_Close            77 non-null     float64
 4   Previous_Adj_Close        77 non-null     float64
 5   Previous_Volume           77 non-null     float64
 6   Previous_Search_Interest  77 non-null     float64
 7   isPartial                 77 non-null     object 
 8   Open                      77 non-null     float64
 9   High                      77 non-null     float64
 10  Low                       77 non-null     float64
 11  Close                     77 non-null     float64
 12  Adj Close                 77 non-null     float64
 13  Volume                    77 non-null     float

In [5]:
columns = ["Previous_Open", "Previous_High", "Previous_Low", "Previous_Close", "Previous_Volume", "Previous_Search_Interest", "Close"]
mergedPrice = mergedPrice[columns]
mergedPrice

,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest,Close
2021-04-01,646.619995,672.000000,641.109985,667.929993,33337300.0,71.0,661.750000
2021-04-05,688.369995,692.419983,659.419983,661.750000,35298400.0,76.0,691.049988
2021-04-06,707.710022,708.159973,684.700012,691.049988,41842800.0,82.0,691.619995
2021-04-07,690.299988,696.549988,681.369995,691.619995,28271800.0,69.0,670.969971
2021-04-08,687.000000,691.380005,667.840027,670.969971,26309400.0,64.0,683.799988
...,...,...,...,...,...,...,...
2021-07-15,670.750000,678.609985,652.840027,653.380005,21641200.0,57.0,650.599976
2021-07-16,658.390015,666.140015,637.880005,650.599976,20209600.0,59.0,644.219971
2021-07-19,654.679993,656.700012,642.200012,644.219971,16339800.0,62.0,646.219971
2021-07-20,629.890015,647.200012,621.289978,646.219971,21297100.0,65.0,660.500000


In [6]:
features = list(mergedPrice.columns)
features.remove("Close")
target = "Close"

X = mergedPrice[features]
y = mergedPrice[target]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, shuffle=False)

In [8]:
lr = LinearRegression()
lr

LinearRegression()

In [9]:
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
lr.score(X_train, y_train)

0.889909218635416

In [11]:
lr.score(X_test, y_test)

0.12337453509099794

In [12]:
def printMetrics(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

In [13]:
predictions = lr.predict(X_test)
printMetrics(y_test, predictions)

Score: 0.13
MAE: 9.64
RMSE: 12.00
r2: 0.12


In [14]:
predictedPrice = pd.concat([X_test, y_test], axis=1)
predictedPrice = predictedPrice.rename(columns={'Close': 'Predicted_Close'})
predictedPrice

,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest,Predicted_Close
2021-07-01,679.770020,692.809998,678.140015,679.700012,18924900.0,64.0,677.919983
2021-07-02,683.919983,687.989990,672.799988,677.919983,18634500.0,63.0,678.900024
2021-07-06,678.979980,700.000000,673.260010,678.900024,27054500.0,64.0,659.580017
2021-07-07,681.710022,684.000000,651.400024,659.580017,23284500.0,64.0,644.650024
2021-07-08,664.270020,665.700012,638.320007,644.650024,18792000.0,57.0,652.809998
2021-07-09,628.369995,654.429993,620.460022,652.809998,22773300.0,63.0,656.950012
2021-07-12,653.179993,658.909973,644.690002,656.950012,18118500.0,64.0,685.700012
2021-07-13,662.200012,687.239990,662.159973,685.700012,25927000.0,65.0,668.539978
2021-07-14,686.320007,693.280029,666.299988,668.539978,20966100.0,60.0,653.380005
2021-07-15,670.750000,678.609985,652.840027,653.380005,21641200.0,57.0,650.599976
